In [ ]:
""" Activate if you don't have those package, optuna, lightgbm """
!pip install optuna
!pip install lightgbm

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
import warnings
import logging
import os, sys, gc, random
from sklearn.metrics import mean_squared_error

warnings.simplefilter("ignore")
logging.disable(logging.ERROR)

In [ ]:
""" Helper Function """
def load_data(data_path: str) -> pd.DataFrame:
    """ Load data_folder from csv file like as train.csv, test.csv, val.csv """
    df = pd.read_csv(data_path)
    return df

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }

def compt_score(content_true, content_pred, wording_true, wording_pred):
    content_score = mean_squared_error(content_true, content_pred)**(1/2)
    wording_score = mean_squared_error(wording_true, wording_pred)**(1/2)
    
    return (content_score + wording_score)/2


def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(seed=42)


In [ ]:
""" Load Out of Fold Train Dataframe and Check Features of Dataframe """

train = load_data('oof_train 경로 입력')
train

In [ ]:
""" Feature Selection for LightGBM, Optuna """

targets = ["content", "wording"]

drop_columns = ["fold", "student_id", "prompt_id", "text", "fixed_summary_text",
                "prompt_question", "prompt_title", 
                "prompt_text"] + targets

In [ ]:
""" Optuna Hyper-Params Tuning for LGBM """

def objective(trial, X_train_cv, y_train_cv, X_eval_cv, y_eval_cv):
    dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
    dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)
    max_depth = trial.suggest_int('max_depth', 2, 8)
    params = {
        'boosting_type': 'gbdt',
        'random_state': 42,
        'objective': 'regression',
        'metric': 'rmse',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': max_depth,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 2**max_depth - 1),
        'verbosity': -1  # Add this line to suppress warnings and info messages
    }

    evaluation_results = {}
    model = lgb.train(params,
                      num_boost_round=10000,
                      valid_names=['train', 'valid'],
                      train_set=dtrain,
                      valid_sets=dval,
                    #   verbose_eval=1000,
                      callbacks=[
                              lgb.early_stopping(stopping_rounds=30, verbose=True),
                              lgb.log_evaluation(100),
                              lgb.callback.record_evaluation(evaluation_results),
                        ],)

    # Use the last metric for early stopping
    evals_result = model.best_score
    last_metric = list(evals_result.values())[-1]
    trial.set_user_attr('best_model', model)  # Save the model in the trial
    return last_metric[list(last_metric.keys())[-1]]

model_dict = {}

for target in targets:
    models = []
    
    for fold in range(4):
        X_train_cv = train[train["fold"] != fold].drop(columns=drop_columns)
        y_train_cv = train[train["fold"] != fold][target]

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective(trial, X_train_cv, y_train_cv, X_eval_cv, y_eval_cv), n_trials=100)
        
        print('Best trial: score {}, params {}'.format(study.best_value, study.best_params))
        best_model = study.trials[study.best_trial.number].user_attrs['best_model']
        models.append(best_model)
    
    model_dict[target] = models

In [ ]:
""" validation of oputna result """


model_dict = {}

for target in targets:
    models = []
    
    for fold in range(4):
        X_train_cv = train[train["fold"] != fold].drop(columns=drop_columns)
        y_train_cv = train[train["fold"] != fold][target]

        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        dtrain = lgb.Dataset(X_train_cv, label=y_train_cv)
        dval = lgb.Dataset(X_eval_cv, label=y_eval_cv)
        
        # input params from optuna result
        params = {
            'boosting_type': 'gbdt',
            'random_state': 42,
            'objective': 'regression',
            'metric': 'rmse',
            'learning_rate': 0.036873986853722646,
            'max_depth': 3,  #3
            'lambda_l1': 0.00010854204935611854,
            'lambda_l2': 2.8816343485123506e-05,
            'num_leaves': 4
        }

        evaluation_results = {}
        model = lgb.train(params,
                          num_boost_round=10000,
                            #categorical_feature = categorical_features,
                          valid_names=['train', 'valid'],
                          train_set=dtrain,
                          valid_sets=dval,
                          callbacks=[
                              lgb.early_stopping(stopping_rounds=30, verbose=True),
                              lgb.log_evaluation(100),
                              lgb.callback.record_evaluation(evaluation_results)
                          ],
                         )
        models.append(model)
    
    model_dict[target] = models

In [ ]:
# cv
rmses = []

for target in targets:
    models = model_dict[target]

    preds = []
    trues = []
    
    for fold, model in enumerate(models):
        X_eval_cv = train[train["fold"] == fold].drop(columns=drop_columns)
        y_eval_cv = train[train["fold"] == fold][target]

        pred = model.predict(X_eval_cv)

        trues.extend(y_eval_cv)
        preds.extend(pred)
        
    rmse = np.sqrt(mean_squared_error(trues, preds))
    print(f"{target}_rmse : {rmse}")
    rmses = rmses + [rmse]

print(f"mcrmse : {sum(rmses) / len(rmses)}")